In [129]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
import datetime
import pandas as pd
import lxml
import re

In [130]:
columns = ["id", "title", "subtitle", "content", "release_date", "solved_by_count", "difficulty"]

def getFullProblemSet():
    curr_problem = 1
    url = "https://projecteuler.net/problem=1"

    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)

    browser.visit(url)

    #add a delay so page fully loads
    time.sleep(4)

    problems = []
    df = pd.DataFrame(columns=columns)

    while(browser.url.find("archive") == -1):
        print(curr_problem)
        current_problem = browser.html
        
        try:
            news_soup = bs(current_problem, "html.parser")

            title = news_soup.find_all("h2")[0].get_text()
            subtitle = news_soup.find_all("h3")[0].get_text()
            content = news_soup.find_all("div", class_="problem_content")[0].get_text()
            tooltip = news_soup.find_all(class_="tooltiptext_right")[0].get_text()

            # split tooltip text into separate features
            # datetime (may need to set UTC as timezone? Or a different timezone?)
            datetime_str = re.findall("Published on (.*[ap]m);", tooltip)
    #         print(datetime_str)
            datetime_str = re.sub("(?<=\d)[a-z][a-z]", "", datetime_str[0])
            datetime_str = re.sub("(?<= )(\d)(?= )", "0\g<0>", datetime_str)
            datetime_str = datetime_str.replace("pm", "PM")
            datetime_str = datetime_str.replace("am", "AM")
    #         print(datetime_str)
            datetime_val = datetime.datetime.strptime(datetime_str, "%A, %d %B %Y, %I:%M %p")

            # Solved By (count)
            solved_by_count = re.findall("Solved by (\d+)", tooltip)

            # Difficulty Rating
            difficulty = re.findall("(\d+)%", tooltip)
            if(len(difficulty) == 0):
                difficulty = [float('nan')]

            curr_row = pd.Series({"id":curr_problem, "title":title, "subtitle":subtitle, "content":content, "release_date":datetime_val.isoformat(), "solved_by_count":int(solved_by_count[0]), "difficulty":int(difficulty[0])})
            df = pd.concat([df, curr_row.to_frame().T], ignore_index=True)
#         print(curr_row)
        except:
            pass
        curr_problem += 1
        
        url = "https://projecteuler.net/problem={}".format(curr_problem)
        browser.visit(url)
        time.sleep(4)

    browser.quit()
    
    return df


In [131]:


test_df = getFullProblemSet()
test_df

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,id,title,subtitle,content,release_date,solved_by_count,difficulty
0,1,Multiples of 3 or 5,Problem 1,\nIf we list all the natural numbers below 10 ...,2001-10-05T18:00:00,969832,5
1,2,Even Fibonacci numbers,Problem 2,\nEach new term in the Fibonacci sequence is g...,2001-10-19T18:00:00,774117,5
2,3,Largest prime factor,Problem 3,"\nThe prime factors of 13195 are 5, 7, 13 and ...",2001-11-02T18:00:00,557641,5
3,4,Largest palindrome product,Problem 4,\nA palindromic number reads the same both way...,2001-11-16T18:00:00,493738,5
4,5,Smallest multiple,Problem 5,\n2520 is the smallest number that can be divi...,2001-11-30T18:00:00,497077,5
...,...,...,...,...,...,...,...
817,818,SET,Problem 818,\n\nThe SET® card game is played with a pack o...,2022-11-27T10:00:00,93,85
818,819,Iterative Sampling,Problem 819,\nGiven an n-tuple of numbers another n-tuple ...,2022-12-03T13:00:00,179,45
819,820,Nth digit of Reciprocals,Problem 820,\nLet dn(x) be the nth decimal digit of the fr...,2022-12-10T16:00:00,507,10
820,821,123-Separable,Problem 821,"\n\nA set, S, of integers is called 123-separa...",2022-12-17T19:00:00,105,65


In [126]:
test_df

,id,title,subtitle,content,release_date,solved_by_count,difficulty
0,1,Multiples of 3 or 5,Problem 1,\nIf we list all the natural numbers below 10 ...,2001-10-05T18:00:00,969832,5
1,2,Even Fibonacci numbers,Problem 2,\nEach new term in the Fibonacci sequence is g...,2001-10-19T18:00:00,774117,5
2,3,Largest prime factor,Problem 3,"\nThe prime factors of 13195 are 5, 7, 13 and ...",2001-11-02T18:00:00,557641,5
3,4,Largest palindrome product,Problem 4,\nA palindromic number reads the same both way...,2001-11-16T18:00:00,493737,5


In [119]:
test_df.to_csv("./3-11-23.csv", index=False)

# Outline of PE Scraping Process

- navigate to PE problem 1 (https://projecteuler.net/problem=1)
- scrape webpage from browser using beautifulsoup
- add problem information to dataframe
- increment problem number
- repeat navigation, scrape, and data append for each problem until returned to the main archives page (https://projecteuler.net/archives)
- output data to a file (.csv)
- should take ~1 hour for 800 problems (given a 5 second page load time)

## To Do
- Fix problem descriptions that use MathJax (https://docs.mathjax.org/en/v1.1-latest/mathjax.html) notation (highest priority because it directly affects problem description clarity and readability, which directly impacts the dataset's quality as a problem repository)
- Add data files for problems that use external files (like Poker Hands)
- Add links for problems that use external information (like Roman Numerals)
- Add a notebook to update the problem set file (only add new problems since older problems do not change often)
- Can also rescrape entire notebook since number of solvers for a problem should increase over time? (may want to add versioning using a date/timestamp, or can just use git)